# Introduction
I've noticed that the biomass reaction in the model doesn't match what is present in Beata's thesis. Tables 4.1, 4.2, 4.3, 4.4, 4.5 and 4.6 cover results of experimental determination of the biomass components. The biomass reaction should encompass all of these metabolites. I have noticed that quite some metabolites are missing, or that the coefficients do not exactly match the presented data. So here I will try to make the reaction match the data presented. I will go per category, and after adding each missing metabolite check biomass formation is still possible. If not, I will try to find out the root cause and tackle that problem. 

All the way at the end, when the model is mass and stoiciometrically balanced we can fit the GAM to the data presented in the thesis; for now I will leave it as it is. 

In [5]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [6]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [7]:
model_e_coli = cameo.load_model ('iML1515')

In [8]:
model_b_sub = cameo.load_model ('iYO844')

In [5]:
#available at: https://github.com/SysBioChalmers/yeast-GEM/blob/master/ModelFiles/xml/yeastGEM.xml
model_yeast = cobra.io.read_sbml_model('../../Databases/yeastGEM.xml')

In [6]:
model.optimize().objective_value

1.8130259343813175

## Ions (table 4.1)
Table 4.1 shows the determination of the following ions: K, Mg, Fe3+, Ca, phosphate and diphosphate. Note that the units of the metabolites added should be mmol/gdcw. 

Our reaction currently has the ions: fe3, fe2 and cl. I don't know why or where they were added, but I will remove them. Here I will try to stringently match the data presented.

Note: phosphate is also formed from the hydrolysis of ATP, so this number will need to change when the GAM is estimated. 

In [12]:
#Potassium
model.reactions.biomass.add_metabolites({
    model.metabolites.k_c: -0.7082
})

In [13]:
#we also supply potassium in the minimal medium, so add the exchange
model.add_boundary(model.metabolites.k_e, type= 'exchange',reaction_id = 'EX_k_e',lb = -1000, ub = 1000)

Reaction identifier,EX_k_e
Name,Potassium exchange
Memory address,0x020e4230d708
Stoichiometry,k_e <=> Potassium <=>
GPR,
Lower bound,-1000
Upper bound,1000


In [14]:
#magnesium needs to be added to the model first
model.add_metabolites(Metabolite(id='mg2_c'))
model.add_metabolites(Metabolite(id='mg2_e'))

In [15]:
model.metabolites.mg2_c.name = 'Magnesium'
model.metabolites.mg2_c.compartment= 'c'
model.metabolites.mg2_c.formula = 'Mg'
model.metabolites.mg2_c.annotation = model_e_coli.metabolites.mg2_c.annotation
model.metabolites.mg2_c.charge = 2

In [16]:
model.metabolites.mg2_e.name = 'Magnesium'
model.metabolites.mg2_e.compartment= 'e'
model.metabolites.mg2_e.formula = 'Mg'
model.metabolites.mg2_e.annotation = model_e_coli.metabolites.mg2_c.annotation
model.metabolites.mg2_e.charge = 2

In [17]:
#the mg channel has been found in geobacillus, so will add passive transport.
model.add_reaction(Reaction(id='MG2t'))

In [18]:
model.reactions.MG2t.name = 'Magnesium transport via channel'
model.reactions.MG2t.annotation['sbo'] = 'SBO:0000185'
model.reactions.MG2t.bounds = (-1000,1000)

In [19]:
model.reactions.MG2t.add_metabolites({model.metabolites.mg2_c: -1, model.metabolites.mg2_e: 1})

In [20]:
#add exchange
model.add_boundary(model.metabolites.mg2_e, type= 'exchange',reaction_id = 'EX_mg2_e',lb = -1000, ub = 1000)

Reaction identifier,EX_mg2_e
Name,Magnesium exchange
Memory address,0x020e424a9588
Stoichiometry,mg2_e <=> Magnesium <=>
GPR,
Lower bound,-1000
Upper bound,1000


In [21]:
#add mg to biomass
model.reactions.biomass.add_metabolites({
    model.metabolites.mg2_c: -0.098
})

In [22]:
#Fe3+ is already in the model, just need to change unit
model.reactions.biomass.add_metabolites({
    model.metabolites.fe3_c:-0.0041
})

In [23]:
#calcium needs to be added to the model first
model.add_metabolites(Metabolite(id='ca2_c'))
model.add_metabolites(Metabolite(id='ca2_e'))

In [24]:
model.metabolites.ca2_c.name = 'Calcium'
model.metabolites.ca2_c.compartment= 'c'
model.metabolites.ca2_c.formula = 'Ca'
model.metabolites.ca2_c.annotation = model_e_coli.metabolites.ca2_c.annotation
model.metabolites.ca2_c.charge = 2

In [25]:
model.metabolites.ca2_e.name = 'Calcium'
model.metabolites.ca2_e.compartment= 'e'
model.metabolites.ca2_e.formula = 'Ca'
model.metabolites.ca2_e.annotation = model_e_coli.metabolites.ca2_e.annotation
model.metabolites.ca2_e.charge = 2

In [26]:
#there is a calcium antiporter annotated in the genome
model.add_reaction(Reaction(id='CA2t'))

In [27]:
model.reactions.CA2t.name = 'Transport of calcium via antiport'
model.reactions.CA2t.annotation['sbo'] = 'SBO:0000185'
model.reactions.CA2t.bounds = (-1000,1000)

In [28]:
model.reactions.CA2t.add_metabolites({model.metabolites.ca2_c: -1, model.metabolites.ca2_e:1, model.metabolites.h_e: -1, model.metabolites.h_c:1})

In [29]:
#add exchange
model.add_boundary(model.metabolites.ca2_e, type= 'exchange',reaction_id = 'EX_ca2_e',lb = -1000, ub = 1000)

Reaction identifier,EX_ca2_e
Name,Calcium exchange
Memory address,0x020e42170b08
Stoichiometry,ca2_e <=> Calcium <=>
GPR,
Lower bound,-1000
Upper bound,1000


In [30]:
#add ca to biomass
model.reactions.biomass.add_metabolites({
    model.metabolites.ca2_c: -0.00315
})

__phosphate__ is orginally in the biomass reaction already: 104.9856 (coupled to the amount of ATP required for growth. 
As a substrate that is consumed, phosphate only needs 0,0173 mmol/gCDW. This will play only a small difference on the total reaction. I will not change this yet, but this should be modified when we fit the GAM. 

In [31]:
#diphosphate
model.reactions.biomass.add_metabolites({model.metabolites.ppi_c:-0.0012})

In [32]:
#remove fe2 and cl from the biomass reaction
model.reactions.biomass.add_metabolites({model.metabolites.fe2_c:0.029903042, model.metabolites.cl_c:0.029903042})

In [33]:
#save & commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [34]:
model.optimize().objective_value

1.7870624929522956

## Amino acids (table 4.2)
I've already ensured all amino acids are present in the model. However, the coefficients associated to a majority of them do not fit what is included in table 4.2, so I will modify those where necessary to match the table.

In [35]:
model.reactions.biomass.add_metabolites({
    model.metabolites.gly_c:-0.02407042, 
    model.metabolites.ala__L_c:-0.0303495, 
    model.metabolites.val__L_c:-0.02220057, 
    model.metabolites.leu__L_c:-0.02051081, 
    model.metabolites.ile__L_c:-0.0161025,
    model.metabolites.ser__L_c:-0.01068746,
    model.metabolites.thr__L_c:-0.0158668,
    model.metabolites.phe__L_c:-0.00856894,
    model.metabolites.tyr__L_c:0.060075203,
    model.metabolites.trp__L_c:-0.00605801,
    model.metabolites.cys__L_c:-0.00636758,
    model.metabolites.met__L_c:-0.00301383,
    model.metabolites.lys__L_c:-0.018717862,
    model.metabolites.arg__L_c:-0.01105,
    model.metabolites.his__L_c:-0.00459031,
    model.metabolites.asp__L_c:-0.01310304,
    model.metabolites.glu__L_c:-0.250734713,
    model.metabolites.pro__L_c: -0.00983182,
    model.metabolites.gln__L_c:0.322355,
    model.metabolites.asn__L_c:0.18114312
})


In [36]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [37]:
model.optimize().objective_value

1.8146393014541327

## DNA & RNA (table 4.3)
Here table 4.3 estimates values around 0.2-0.3 per nucleotide, or even 8 mmol/gcdw for UMP and CMP. This is a factor 10 higher than the information about the DNA in figure 4.6. This makes it hard to decide which value to use, though for DNA I would expect values near 0.02 mmol/gCDW, and for RNA around 0.15 mm/gDCW.

As the data is hard to really interpret and the outcome of it seems strange. E.g. From the DNA distribution in fig 4.6 we would have 75.4% GC content in the strain, but in the genome we know this should be around 43% (allowing some experimental error). 

Niko mentioned that we could alternatively estimate the DNA and RNA composition by knowing the GC-content of the genome and the total amounts of DNA and RNA in our bacteria. So instead I will estimate the values from this.

GC-content: approx 43.4 mol%
total DNA (1%) i.e. 1g DNA / 100 gDCW
total RNA (16% of biomass) i.e. 16 g / 100 gDCQ

From this data, together with the MW of each (d)NMP we can estimate the mmol/gDCW of each nucleotide present. see '../databases/Biomass_rct.xlsx' for the calculations and desired final values.

I will correct the values in the model here.

In [39]:
model=cobra.io.read_sbml_model('../model/g-thermo.xml')

In [40]:
model.reactions.biomass.add_metabolites({
    model.metabolites.damp_c:0.021231612,
    model.metabolites.dcmp_c: 0.016684519,
    model.metabolites.dtmp_c: 0.021231612,
    model.metabolites.dgmp_c: 0.016684519,
    model.metabolites.amp_c:-0.020399789,
    model.metabolites.gmp_c: 0.043519524,
    model.metabolites.cmp_c:-0.022435461,
    model.metabolites.ump_c:-0.046781783,
})

In [43]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

## Carbohydrates
Beata's thesis identifies that 10% of DCW is composed of carbohydrates. One would also expect these to be present in the biomass. In the thesis, the data shows the carbohydrate component as split into basic sugars which is convenient from a modelling perspective. It may influence the energy requirement but this will be fit in the GAM anyway.

In [89]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [90]:
model.reactions.biomass.add_metabolites({
    model.metabolites.arab__L_c: -0.045625,
    model.metabolites.gal_c:-0.212236,
    model.metabolites.glc__D_c:-0.010725,
    model.metabolites.xyl__D_c: -0.204370,
    model.metabolites.man_c:-0.006411,
    model.metabolites.fru_c:-0.117367
})

Adding the carbohydrates kills the biomass accumulation: so somehow we cannot produce one (or more) of the carbohydrates. 

The problematic carbohydrates are: arabinose, galactose, xylose and mannose. Here I will go through their biosynthesis one by one to ensure they can be made and fix this issue.

__Arabinose__ 
in the model, the arabinose can be made from ribulose. However ribulose cannot be made, so I will fix this. This is because the RBK_L1 reaction is irreversible as it consumes ATP. The options we have hear are either to make this reaction reversible and risk it results in the formation of ATP. Or we add a hydrolysis reaction that hydrolysis the Ru5p__L to rbl, which is used for biomass. 

I think this second option is better. We prevent extensive arabinose production for ATP production purposes in this way. Also the different in ATP requirement for the arabinose needed in growth will be captured in the fitted GAM anyway. The hydrolysis reaction will also be irreversible. By both being irreversible it will prevent a cycle as ATP consumption is coupled to one reaction.

In [91]:
model.add_reaction(Reaction(id='RU5PHY'))

In [92]:
model.reactions.RU5PHY.name = 'Hydrolysis of L-Ribulose-5-phosphate'
model.reactions.RU5PHY.notes['NOTES'] = 'Added reaction to allow biomass formation. ATP is captured in GAM'
model.reactions.RU5PHY.annotation['sbo'] = 'SBO:0000176'

In [93]:
model.reactions.RU5PHY.add_metabolites({
    model.metabolites.ru5p__L_c: -1,
    model.metabolites.rbl__L_c: 1, 
    model.metabolites.h2o_c: -1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c: 1
})

__Galactose__ Here the problem is the conversion from gal1p to galatose, captured in the GALKr reaction. In E. coli this reaction is reversible, even though it is associated with atp. Based on thermodynamic prediction, this reaction can be reversible still. of course SLP is possible, we just need to be sure that there is not a surplus of energy created by this reaction. As galactose is not exported out of the cell, I doubt this would not happen and so we will make the reaction reversible again. 

In [94]:
model.reactions.GALKr.bounds=(-1000,1000)

In [95]:
model.reactions.MELIBHY.id = 'GALS3'

__Xylose__ Here again the problem is the formation of xylulose from xylulose-5-phosphate. here based on thermodynamics I would not expect a reversible reaction. So instead, we again will add a hydrolysis of xu5p__D_c reaction to allow biomass consumption. Again the possible difference in ATP needed to make biomass will be reflected in the biomass GAM and so is not an issue here.

In [96]:
model.add_reaction(Reaction(id='XU5PHY'))

In [97]:
model.reactions.XU5PHY.name = 'Hydrolysis of L-Xylulose-5-phosphate'
model.reactions.XU5PHY.notes['NOTES'] = 'Added reaction to allow biomass formation. ATP is captured in GAM'
model.reactions.XU5PHY.annotation['sbo'] = 'SBO:0000176'

In [98]:
model.reactions.XU5PHY.add_metabolites({
    model.metabolites.xu5p__D_c: -1,
    model.metabolites.xylu__D_c: 1, 
    model.metabolites.h2o_c: -1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c: 1
})

__Mannose__ Here we are again missing a reaction that can convert the mannose-6-phosphate back into mannose. The reaction going to man6p is still mass imbalanced and will be fixed later. Based on the fix for that, it is unreasonable to expect a reverse reaction here, and so I will add a hydrolysis reaction again. (Note: same is done in the E. coli model here.)


In [99]:
model.add_reaction(Reaction(id='M6PHY'))

In [100]:
model.reactions.M6PHY.name = 'Hydrolysis of Mannose-6-phosphate'
model.reactions.M6PHY.notes['NOTES'] = 'Added reaction to allow biomass formation. ATP is captured in GAM'
model.reactions.M6PHY.annotation['sbo'] = 'SBO:0000176'

In [101]:
model.reactions.M6PHY.add_metabolites({
    model.metabolites.man6p_c: -1,
    model.metabolites.man_c: 1, 
    model.metabolites.h2o_c: -1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c: 1
})

In [102]:
model.reactions.M6PHY.check_mass_balance()

{}

In [105]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

# Lipids
To include the lipids in the biomass equation, we need to do a significant amount of additional work. We need to first decide which approach we will use to capture lipid metabolism and then update the model to account for the production of all components we need for that approach. Therefore this is a much larger issue in itself, and so will be tackled in another notebook.

# Cofactors
According to Beata's thesis, there should also be NAD, NADP and NADPH included in the model, as well as quinol. For these cofactors there are experimental values available and so this should be adapted in the biomass reaction.

In the model currently, we also have ribflv, coA and fad present. It could be expected that these are required in the formation of biomass. However their stoichiometric coefficients are very high. After meeting with Ben, we discussed a way to get more realistic coefficients for these components: looking into the yeast, e. coli and b. sub models for the values they have and using those as quidelines. Of course, ideally one would have experimental data on this, however this is unavailable, so we can approach it like this. As these numbers are very small and play only a slight role in biomass formation, slight deviations will not show a huge effect and so it is oke to include it like this.

Please se '../databases/Biomass_rct.xlsx' for a summary of the values used for all cofactors.
For NADH, FAD, ribflav and CoA I will use the values of the e. coli model. There is still quite some variance that is observed bbetween the e. coli and yeast model. But the values are often of similar magnitude. As our organism is a bacteria, I will use the e. coli values. 


In [8]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [7]:
model.reactions.biomass.add_metabolites({
    model.metabolites.nad_c: 0.013703042,
    model.metabolites.nadh_c: -0.000045,
    model.metabolites.nadp_c: 0.025203042,
    model.metabolites.nadph_c: 0.026903042,
    model.metabolites.fad_c: 0.029680042,
    model.metabolites.qh2_c: 0.029603042,
    model.metabolites.ribflv_c: 0.029680042,
    model.metabolites.coa_c: 0.029735042
})

In [146]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

# Other
Aside from the above categories, there are also some other metabolites that should be added or looked into. In Beata's thesis there is data about 10-Formyltetrahydrofolate presence that is currently lacking in the biomass reaction. Here i will add this back in.

Also for the accoa_c, succoa_c, chor_c, thmpp_c, gthrd_c and ptrc_c metabolites I will check the stoichiometry that is associated to them by comparing it to the other models that are present and adapt them accordingly as they seem way too high for our model currently. 

Again, here I will use the E. coli data as a guide, as those numbers seem more realistic than what has been added into our model.

In [12]:
model=cobra.io.read_sbml_model('../model/g-thermo.xml')

In [10]:
model.reactions.biomass.add_metabolites({
    model.metabolites.get_by_id('10fthf_c'): -0.0004,
    model.metabolites.accoa_c:0.029624042,
    model.metabolites.chor_c:0.029680042,
    model.metabolites.gthrd_c:0.029680042,
    model.metabolites.ptrc_c: -0.003366958,
    model.metabolites.succoa_c:0.03,
    model.metabolites.thmpp_c:0.030126042
})

In [11]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

By adding 10fthf_c, the model no longer grows. So now I will check how to resolve this bloackage here. If you supply folate to the model, it grows again so the issue lies somewhere in the folate biosynthesis pathway.

Folate can be made from chorismate or GTP. Supplying either of these doesn't rescue biomass production and so we need to go through both pathways completely to make sure they are correct.

Aside from some reactions that are reverisible that shouldn't be, the part leading to 4-aminobenzoate seems to be correct. So it seems it must be further upstream of the 6hmhpt_c metabolite.

Also the pathway from GTP Seems to be correct, so there must be some co-factor or side meetabolite that is the issue blocking this pathway.

In [14]:
model.reactions.DHFR.bounds = (-1000,0)

In [15]:
model.reactions.HMHPTMT.bounds = (0,1000)

Problem is the conversion to form 6hmhpt_c, as supplying dhnpt_c doesnt rescue biomass formation. In the DHNPA2r reaction, the only other involved metabolite is glycoaldehyde. So possibly this cannot be converted sufficiently and so blocks the pathway. Adding the removal of glycoalehyde restores biomass so this is the issue we face currently.
I will check the glycoaldehyde consumption here.

In [16]:
#seems to miss the GCALDD reaction
model.add_reaction(Reaction(id='GCALDD'))

In [17]:
model.reactions.GCALDD.name = 'Glycolaldehyde dehydrogenase'
model.reactions.GCALDD.annotation = model_e_coli.reactions.GCALDD.annotation

In [18]:
model.reactions.GCALDD.add_metabolites({
    model.metabolites.gcald_c:-1,
    model.metabolites.h2o_c: -1,
    model.metabolites.nad_c:-1, 
    model.metabolites.glyclt_c:1,
    model.metabolites.h_c: 2,
    model.metabolites.nadh_c:1
})

In [19]:
#GLYCLTDx and GLYCNOR are duplicate reactions
model.remove_reactions(model.reactions.GLYCNOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [20]:
#problem is: glx can only be converted back into glyclt.

In [21]:
#MALS in the glyoxylate cycle is in the wrong direction!
model.reactions.MALS.bounds=(-1000,0)

The Bacillus genome encodes a Glyoxalate carboligase. A tBlastn search shows that there is a significant hit in our strain. For that reason, it can be expected that this enzyme is also present in our strain.  Hence I will add the reaction here.

In [22]:
model.add_reaction(Reaction(id='GLXCBL'))

In [23]:
model.reactions.GLXCBL.name= 'Glyoxalate carboligase'
model.reactions.GLXCBL.annotation = model_b_sub.reactions.GLXCBL.annotation

In [24]:
model.reactions.GLXCBL.add_metabolites({
    model.metabolites.glx_c: -2,
    model.metabolites.h_c: -1,
    model.metabolites.hop_c:1,
    model.metabolites.co2_c:1
})

In [25]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

# Fix Succo and thmpp
Ben spotted that our succoa and thmpp metabolites ended up with positive coefficients, which they should not have. So here I will quickly fix this.

In [12]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [13]:
model.reactions.biomass.add_metabolites({
    model.metabolites.succoa_c: -1.95E-04,
    model.metabolites.thmpp_c:-0.000446
})

In [14]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

# Carbohydrates
So apparently I miss understood Ben; in the biomass reaction it is best to include the UDP-bound variant of the carbohydrates where that makes sense. This is because adding the carbohydrates in itself can lead to gluconeogenesis being triggered in the model to meetbiomass requirements.Also, for example for glycogen, it is known that this is made by assembling UDP-glucose, not free glucose. Then the energy accounting for polymerization is better and more correctly captured. (i.e. add the activated sugars)

Note: with this, we will leave the stoichiometry the same. because the data as a weight percentage was determined based on just the glucose, not UDP-glucose. so I will not modify these values. As UDP will be released later this will correct for the additional mass that is brought into the reaction. 

In general, the synthesis of the activated sugars comes from a pyrophosphorylase, that takes the NTP and reacts with the sugar-1-phosphate to give the the NDP-sugar moeity that is used for the polymers. So according to this I will add these variants into the biomass reaction.

So what to do with each carb:
- glucose: becomes UDP-glucose, from g1p. 
- Galactose: from UDP-gal, from gal1p or udpg
- Arabinose: from UDP-arabinose via ara1p and udp-xylose
- xylose: from UDP-xylose via xylose-1-phosphate, or from UDP arabinose or UDPglucoronate (udpglcur_c)
- mannose: polymer is mannan. make from mannose-1-phosphate to GDP-mannose and then consume GDP mannose.

- fructose: polymer is inulin. the genes for the production of inulin seem to be missing. So for fructose I will just leave fructose in the reaction, as per bens recommendation.


In [9]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [10]:
#for UDP-arbinose, add UDP-arabinose metabolite and synthesis
model.add_metabolites(Metabolite(id='udparab_c', name = 'UDP-L-arabinose', compartment = 'c', charge = 0, formula = 'C14H22N2O16P2'))

In [11]:
model.metabolites.udparab_c.annotation['kegg.compound'] = 'C00935'
model.metabolites.udparab_c.annotation['chebi'] = 'CHEBI:17983'
model.metabolites.udparab_c.annotation['metanetx.chemical'] = 'MNXM5976' 

In [12]:
#add arabinose-1-phosphate
model.add_metabolites(Metabolite(id='ara1p_c', name = 'beta-L-Arabinose 1-phosphate', compartment = 'c', charge = 0, formula = 'C5H11O8P'))

In [13]:
model.metabolites.ara1p_c.annotation['kegg.compound'] = 'C03906'
model.metabolites.ara1p_c.annotation['chebi'] = 'CHEBI:15807'

In [14]:
#add reaction from ara1p to udparab
model.add_reaction(Reaction(id='ARAAT', name = 'UDP-sugar pyrophosphorylase '))

In [15]:
model.reactions.ARAAT.annotation['kegg.reaction'] = 'R08845'
model.reactions.ARAAT.annotation['ec-code'] = '2.7.7.64'
model.reactions.ARAAT.annotation['sbo'] = 'SBO:0000176'

In [16]:
model.groups.get_by_id('00520 - Amino sugar and nucleotide sugar metabolism').add_members(model.reactions.ARAAT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



In [17]:
model.reactions.ARAAT.add_metabolites({
    model.metabolites.utp_c:-1,
    model.metabolites.ara1p_c:-1,
    model.metabolites.ppi_c:1,
    model.metabolites.udparab_c:1
})

In [18]:
#add reaction from arabinose to ara1p
model.add_reaction(Reaction(id='ARAK', name = 'L-arabinokinase'))

In [19]:
model.reactions.ARAK.annotation['sbo'] = 'SBO:0000176'
model.reactions.ARAK.annotation['kegg.reaction'] = 'R01754'
model.reactions.ARAK.annotation['ec-code'] = '2.7.1.46'

In [20]:
model.groups.get_by_id('00520 - Amino sugar and nucleotide sugar metabolism').add_members(model.reactions.ARAK)

In [21]:
model.reactions.ARAK.add_metabolites({
    model.metabolites.arab__L_c:-1,
    model.metabolites.atp_c:-1,
    model.metabolites.ara1p_c:1,
    model.metabolites.adp_c:1,
    model.metabolites.h_c:-1
})

In [22]:
#add udp-xylose met 
model.add_metabolites(Metabolite(id='udpxyl_c', name='UDP-D-xylose', compartment ='c', formula ='C14H22N2O16P2', charge=0 ))

In [23]:
model.metabolites.udpxyl_c.annotation['kegg.compound'] = 'C00190'
model.metabolites.udpxyl_c.annotation['chebi'] = 'CHEBI:16082'

In [24]:
#add conversion from UDP-arabinose to UDP.xylose
model.add_reaction(Reaction(id='XYLAT', name = 'UDP-arabinose 4-epimerase'))

In [25]:
model.reactions.XYLAT.bounds = (-1000,1000)
model.reactions.XYLAT.annotation['sbo'] = 'SBO:0000176'
model.reactions.XYLAT.annotation['ec-code'] = '5.1.3.5'
model.reactions.XYLAT.annotation['kegg.reaction'] = 'R01473'

In [26]:
model.groups.get_by_id('00520 - Amino sugar and nucleotide sugar metabolism').add_members(model.reactions.XYLAT)

In [27]:
model.reactions.XYLAT.add_metabolites({
    model.metabolites.udparab_c:-1,
    model.metabolites.udpxyl_c:1
})

In [28]:
#add conversion of xylUDP from UDP-glucuronate.
model.add_reaction(Reaction(id='UDPGDC', name = 'UDP-glucuronate decarboxylase'))

In [29]:
model.reactions.UDPGDC.bounds = (0,1000)
model.reactions.UDPGDC.annotation['sbo'] = 'SBO:0000176'
model.reactions.UDPGDC.annotation['ec-code'] = '4.1.1.35'
model.reactions.UDPGDC.annotation['kegg.reaction'] = 'R01384'

In [30]:
model.groups.get_by_id('00520 - Amino sugar and nucleotide sugar metabolism').add_members(model.reactions.UDPGDC)

In [31]:
model.reactions.UDPGDC.add_metabolites({
    model.metabolites.udpglcur_c:-1,
    model.metabolites.udpxyl_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.h_c:-3
})

In [32]:
model.reactions.UDPGDC.check_mass_balance()

{}

In [33]:
#conversion of udpglcur from udpg is already present.
#model.reactions.UDPGD

In [31]:
#GDP-mannose from man1p: raction MAN1PT
#man1P is coupled to carbon metabolism.

In [35]:
#remove all carbohydrates metabolites from biomass rct
model.reactions.biomass.add_metabolites({
    model.metabolites.arab__L_c: 0.045625,
    model.metabolites.gal_c:0.212236,
    model.metabolites.glc__D_c:0.010725,
    model.metabolites.xyl__D_c:0.204370,
    model.metabolites.man_c:0.006411
})

In [36]:
#add udp/gdp-variant of carbohydrates and expell the udp/gdp too
model.reactions.biomass.add_metabolites({
    model.metabolites.udparab_c: -0.045625,
    model.metabolites.udpgal_c:-0.212236,
    model.metabolites.udpg_c: -0.010725,
    model.metabolites.udpxyl_c: -0.204370,
    model.metabolites.gdpmann_c: -0.006411,
    model.metabolites.udp_c:0.472956, #releasing the UDP, from the total of the above UDP related metabolites
    model.metabolites.gdp_c: 0.006411 #releasing the GDP 
})

In [42]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Also Ben was curious what the current g/gCDW total of our biomass reaction was. So here i will write the script that can calculate that for us.

In [38]:
#check total g/gdcw biomass
biomass_g_g = []
for met in model.reactions.biomass.metabolites:
    stoich = model.reactions.biomass.metabolites.get(met) #get stoichiometry in mmol/gcdw
    if stoich == 1 :#ignore the biomass part, but need to include the ADP/Pi/UDP that is also made here
        continue
    else:
        mw = met.formula_weight # get molecular weight of metabolite in g/mol
        try:
            weight_fraction = (mw/1000)*stoich #the g/gcdw of this metabolite
            biomass_g_g.append(weight_fraction)
        except TypeError: #for the met with R in it, just ignore
            print(met.id)

qh2_c


C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\metabolite.py:130: UserWarning:

The element 'R' does not appear in the periodic table



In [39]:
#to find total g/gcdw of all mets, sum up this list
abs(sum(biomass_g_g))

1.0273777668041792